In [1]:
import pandas as pd
import numpy as np
import OpenBlender
import json
%matplotlib inline
import matplotlib
from pathlib import Path
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [2]:
import nltk
# nltk.download('wordnet')

# import these modules 
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
# nltk.sentiment.vader

ps = PorterStemmer() # stemming
wordnet_lemmatizer = WordNetLemmatizer()  #Lemmatization

In [ ]:
# get sentiment score applying the vaderSentiment functions
file_path = Path('5ea20b4e95162936348f141d.csv')
initial_lite = pd.read_csv(file_path )

In [ ]:
initial_lite.describe()

In [ ]:
initial_lite.drop_duplicates('timestamp', inplace=True )

In [ ]:
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [ ]:
print(emoticons)

In [ ]:
import re
# Emojis pattern
emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

In [ ]:
emoji_pattern

In [ ]:
# punctuations pattern
punctuation_pattern = string.punctuation + "▼"
punctuation_pattern

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords_re = set(STOPWORDS)
update_list = ["btc", "bitcoin","blockchain", "crypto","cryptocurrency","ha","say","time","want","gonna","use","a","about",
              "actually","almost","also","although","always","am","an","and","any","are","as","at","be","became","become",
              "but","by","can","could","did","do","does","each","either","else","for","from","had","has","have","hence","how",
              "I","if","in","is","its","just","may","maybe","me","might","mine","must","my","mine","must","neither","nor","not",
              "of","oh","ok","when","where","whereas","wherever","whenever","whether","which","while","who","whom","whomever","whose",
              "why","will","with","within","without","would","yes","yet","you","your","people"]
stopwords_re.update(update_list)
print(stopwords_re)

In [ ]:
'''
clean_tweets()
contains: remove links, tokenize, reomve non-ASCII chars, convert to lower case, remove stop_words, remove
'''
def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    # remove links
    tweet = re.sub(r'http\S+', '', tweet)
    #after tweepy preprocessing the colon left remain after removing mentions
    #or RT sign in the import re
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    tweet = re.sub(r'([0-9])\w+','',tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
    word_tokens = word_tokenize(tweet)
    #tweet = re.sub(r"\b\w+\b",' ', tweet)
    #filter using NLTK library append it to a string
    #filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
    #looping through conditions
    for w in word_tokens:
        # convert to lower case
        w = w.lower()
        # lemmanization
        w = wordnet_lemmatizer.lemmatize(w)
        #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in punctuation_pattern:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)

In [ ]:
initial_lite['clean_text']=initial_lite['VEC.COINJOURNA.text_last1days:bitcoin'].apply(clean_tweets)

In [ ]:
file_path = Path('Lit_prices_target.csv')
df_anchor_lite=pd.read_csv(file_path )

In [ ]:
df_anchor = pd.concat([df_anchor_lite, initial_lite])

In [ ]:
df_anchor

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from imageio import imread
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os